In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/instacart-market-basket-analysis/departments.csv.zip
/kaggle/input/instacart-market-basket-analysis/sample_submission.csv.zip
/kaggle/input/instacart-market-basket-analysis/order_products__train.csv.zip
/kaggle/input/instacart-market-basket-analysis/order_products__prior.csv.zip
/kaggle/input/instacart-market-basket-analysis/orders.csv.zip
/kaggle/input/instacart-market-basket-analysis/products.csv.zip
/kaggle/input/instacart-market-basket-analysis/aisles.csv.zip


## 1. Data Exploration and Preprocessing
### a. Load the Data:
 - Load the CSV files into pandas DataFrames.
#### Dateframs

    - `departments`
    - `sample_submission`
    - `order_products__train`
    - `order_products__prior` 
    - `orders`
    - `products`
    - `aisles`
 - Check for missing values and data types.

In [3]:
path = "/kaggle/input/instacart-market-basket-analysis/"
files = [
    "departments.csv.zip",
    "sample_submission.csv.zip",
    "order_products__train.csv.zip",
    "order_products__prior.csv.zip",
    "orders.csv.zip",
    "products.csv.zip",
    "aisles.csv.zip"
]
for file in files:
    with zipfile.ZipFile(path+file,"r") as z:
        z.extractall(".")

def read_file(file):
    return pd.read_csv(file)

In [4]:
departments,sample_submission,order_products__train,order_products__prior,orders,products,aisles = \
[read_file(file.split(".zip")[0]) for file in files]


In [5]:
order_products = pd.concat([order_products__train,order_products__prior],ignore_index=True)
orders.drop("eval_set",axis=1,inplace= True)

## New table


- `orders` `drop()` eval_set 
- `order_products` `concat()` order_products__train, order_products__prior
- `departments`
- `sample_submission`
- `products`
- `aisles`

In [6]:
def check_not_null(dataframe):
    return dataframe.notnull().all(axis=1)

def check_data_types(dataframe):
    dataframe.info()

In [7]:
all_dataframe = {"orders":orders,
                 "order_products":order_products,
                 "departments":departments,
                 "sample_submission":sample_submission,
                 "products":products,
                 "aisles":aisles}

In [8]:
for key,df in all_dataframe.items():
    print(f"{key.capitalize()}\n{check_not_null(df).value_counts()}\n")

Orders
True     3214874
False     206209
Name: count, dtype: int64

Order_products
True    33819106
Name: count, dtype: int64

Departments
True    21
Name: count, dtype: int64

Sample_submission
True    75000
Name: count, dtype: int64

Products
True    49688
Name: count, dtype: int64

Aisles
True    134
Name: count, dtype: int64



In [9]:
for key,df in all_dataframe.items():
    print(f"{key.capitalize()}\n")
    check_data_types(df)
    print("\n\n")

Orders

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
dtypes: float64(1), int64(5)
memory usage: 156.6 MB



Order_products

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 1.0 GB



Departments

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         

### b. Data Cleaning:
   - Handle missing values.
   - Convert data types (e.g., datetime conversion).
   - Merge DataFrames as needed (e.g., joining order_products with products).

In [10]:
orders.loc[orders["days_since_prior_order"].isna(),["days_since_prior_order"]] = 100

In [11]:
orders["days_since_prior_order"] = orders["days_since_prior_order"].astype(int)
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                  Dtype
---  ------                  -----
 0   order_id                int64
 1   user_id                 int64
 2   order_number            int64
 3   order_dow               int64
 4   order_hour_of_day       int64
 5   days_since_prior_order  int64
dtypes: int64(6)
memory usage: 156.6 MB


In [12]:
products = products.merge(aisles).merge(departments)

In [13]:
products = products.iloc[:,[0,2,3,1,4,5]]

In [14]:
display(orders.head(),products.head(),order_products.head())

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,100
1,2398795,1,2,3,7,15
2,473747,1,3,3,12,21
3,2254736,1,4,4,7,29
4,431534,1,5,4,15,28


,product_id,aisle_id,department_id,product_name,aisle,department
0,1,61,19,Chocolate Sandwich Cookies,cookies cakes,snacks
1,2,104,13,All-Seasons Salt,spices seasonings,pantry
2,3,94,7,Robust Golden Unsweetened Oolong Tea,tea,beverages
3,4,38,1,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen meals,frozen
4,5,5,13,Green Chile Anytime Sauce,marinades meat preparation,pantry


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


## New table
- `orders`
- `order_products` 
- `products` `merge()` with `aisles` and `departments`

### c. Basic Descriptive Statistics:
   - Calculate summary statistics for numerical features.
   - Examine the distribution of categorical features.

In [15]:
summary_statistics = pd.concat([order_products.iloc[:,[2,3]].describe() , orders.iloc[:,2:6].describe()],axis=1).astype(int)
summary_statistics

,add_to_cart_order,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,33819106,33819106,3421083,3421083,3421083,3421083
mean,8,0,17,2,13,16
std,7,0,17,2,4,22
min,1,0,1,0,0,0
25%,3,0,5,1,10,5
50%,6,1,11,3,13,8
75%,11,1,23,5,16,19
max,145,1,100,6,23,100


In [16]:
categories = products[["department","aisle","product_name"]].sort_values(["department","aisle","product_name"])

In [87]:
categories.groupby(["department","aisle"]).count().join(
    categories.groupby(["department"]).nunique().rename(columns={
    "aisle": "aisle_in_dep",
    "product_name": "products_in_dep"
    })
).rename(columns={
    "product_name": "products_in_aisle",
}).reset_index().set_index(["department","aisle_in_dep", "products_in_dep","aisle"])

products_in_aisle
department aisle_in_dep products_in_dep aisle                                        
alcohol    5            1054            beers coolers                             385
                                        red wines                                 232
                                        specialty wines champagnes                 95
                                        spirits                                   195
                                        white wines                               147
babies     4            1081            baby accessories                           44
                                        baby bath body care                       132
                                        baby food formula                         718
                                        diapers wipes                             187
...                                                                               ...
snacks     11           6264            cookies cakes                             874
                                        crackers                                  747
                                        energy granola bars                       832
                                        fruit vegetable snacks                    356
                                        ice cream toppings                         85
                                        mint gum                                  168
                                        nuts seeds dried fruit                    582
                                        popcorn jerky                             316
                                        trail mix snack mix                        69

[134 rows x 1 columns]

In [86]:
pd.set_option('display.max_row', False)

## 2. Exploratory Data Analysis (EDA)
### a. Customer Behavior:
 - Average number of orders per user.
 - Average time between orders for each user.
 - Number of orders placed by each customer.
 - Customer segments based on purchase frequency.

In [80]:
(orders["order_number"].count() / orders["user_id"].nunique()).astype(int)


16

In [83]:
orders.groupby("user_id").mean().astype(int)

,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
user_id,,,,,
1,1923450,6,2,10,26
2,1768104,8,2,10,21
3,1934104,7,1,16,18
4,1507834,3,4,12,30
5,1849578,3,1,15,29
...,...,...,...,...,...
206205,1439588,2,3,14,37
206206,1813826,34,2,16,5
206207,1786525,9,2,12,19


In [145]:
orders.groupby("user_id")["order_number"].count()

user_id
1         11
2         15
3         13
4          6
5          5
6          4
7         21
8          4
9          4
10         6
11         8
12         6
          ..
206198     8
206199    20
206200    24
206201    33
206202    23
206203     6
206204     5
206205     4
206206    68
206207    17
206208    50
206209    14
Name: order_number, Length: 206209, dtype: int64

In [183]:
freq = orders.groupby("user_id")["days_since_prior_order"].mean().astype(int)
bins = np.arange(freq.min()-1 , freq.max() + 2,4)
segments = pd.cut(freq,bins).reset_index().rename(columns={"days_since_prior_order":"segments"}).sort_values("segments").set_index("segments")
segments

,user_id
segments,
"(0, 4]",55030
"(0, 4]",192747
"(0, 4]",17742
"(0, 4]",178821
"(0, 4]",200112
"(0, 4]",59819
"(0, 4]",40278
"(0, 4]",133642
"(0, 4]",87293


In [180]:
segments.groupby("segments",observed=True).count()

,user_id
segments,
"(0, 4]",2228
"(4, 8]",11853
"(8, 12]",20446
"(12, 16]",23158
"(16, 20]",24682
"(20, 24]",25046
"(24, 28]",24335
"(28, 32]",22522
"(32, 36]",20001


### b. Product Analysis:
 - Identify most popular products by frequency.
 - Determine average order size (number of items per order).

In [212]:
#33819106 
products[products["product_id"] == order_products["product_id"].value_counts().idxmax()]

,product_id,aisle_id,department_id,product_name,aisle,department
24851,24852,24,4,Banana,fresh fruits,produce
